<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf203-dogsvscats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import shutil
import glob

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [2]:
!mkdir ~/.kaggle 

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c dogs-vs-cats

 99% 805M/812M [00:03<00:00, 172MB/s]
100% 812M/812M [00:03<00:00, 219MB/s]


In [5]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [ ]:
!unzip train.zip

In [7]:
!ls

dogs-vs-cats.zip  sample_data		test1.zip  train.zip
kaggle.json	  sampleSubmission.csv	train


In [8]:
filename=glob.glob('train/cat*')
print(len(filename))

12500


In [9]:
if os.path.isdir('valid/dog') is False:
  os.makedirs('valid/dog')
  for c in random.sample(glob.glob('train/dog*'),1000):
    shutil.move(c,'valid/dog')


In [13]:
filename=glob.glob('valid/dog/*')
print(len(filename))
filename=glob.glob('train/dog*')
print(len(filename))

1000
11500


In [11]:
if os.path.isdir('valid/cat') is False:
  os.makedirs('valid/cat')
  for c in random.sample(glob.glob('train/cat*'),1000):
    shutil.move(c,'valid/cat')

In [14]:
filename=glob.glob('valid/cat/*')
print(len(filename))
filename=glob.glob('train/cat*')
print(len(filename))

1000
11500


In [15]:
if os.path.isdir('test/cat') is False:
  os.makedirs('test/cat')
  for c in random.sample(glob.glob('train/cat*'),500):
    shutil.move(c,'test/cat')

In [16]:
filename=glob.glob('test/cat/*')
print(len(filename))
filename=glob.glob('train/cat*')
print(len(filename))

500
11000


In [17]:
if os.path.isdir('test/dog') is False:
  os.makedirs('test/dog')
  for c in random.sample(glob.glob('train/dog*'),500):
    shutil.move(c,'test/dog')

In [19]:
filename=glob.glob('test/dog/*')
print(len(filename))
filename=glob.glob('train/dog*')
print(len(filename))

500
11000


In [20]:
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
  for c in random.sample(glob.glob('train/dog*'),11000):
    shutil.move(c,'train/dog')

In [21]:
if os.path.isdir('train/cat') is False:
  os.makedirs('train/cat')
  for c in random.sample(glob.glob('train/cat*'),11000):
    shutil.move(c,'train/cat')

In [24]:
filename=glob.glob('train/cat/*')
print("Cats in Train ",len(filename))
filename=glob.glob('train/dog/*')
print("Dogs in Train ",len(filename))

Cats in Train  10999
Dogs in Train  10999
